## Introspección

Cómo hacerse preguntas sobre Julia desde Julia.

    Es muy importante para aprender -- trabajar -- documentar

### Estructuras de Datos

In [ ]:
a = 1
typeof(a)

In [ ]:
sizeof(a)

In [ ]:
aa = Int32(1)
sizeof(aa)

In [ ]:
b = 1.
typeof(b)

In [ ]:
sizeof(b)

In [ ]:
c = 1 + im*5.
typeof(c)


In [ ]:
sizeof(c)

In [ ]:
typeof(1//1)

In [ ]:
sizeof(1//1)

In [ ]:
typeof(π)

In [ ]:
A = [1. 2. 6.]
#typeof(A)

In [ ]:
sizeof(A)

In [ ]:
B = [1.; 2. ; 6.]
typeof(B)

In [ ]:
A*B

In [ ]:
typeof(A*B)

In [ ]:
typeof(true)

In [ ]:
typeof(1 < 2)

In [2]:
typeof(1:0.1:4)

StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}

In [5]:
methods(StepRangeLen)

# 5 methods for type constructor:
[1] StepRangeLen(ref::Base.TwicePrecision{T}, step::Base.TwicePrecision{T}, len::Integer) where T in Base at twiceprecision.jl:366
[2] StepRangeLen(ref::Base.TwicePrecision{T}, step::Base.TwicePrecision{T}, len::Integer, offset::Integer) where T in Base at twiceprecision.jl:366
[3] StepRangeLen(r::AbstractRange) in Base at range.jl:1253
[4] StepRangeLen(ref::R, step::S, len::Integer) where {R, S} in Base at range.jl:451
[5] StepRangeLen(ref::R, step::S, len::Integer, offset::Integer) where {R, S} in Base at range.jl:451

In [ ]:
sizeof(1:0.1:4)

In [ ]:
typeof(sin)

### Funciones

En el REPL si escriben `? <ENTER>` verán un nuevo pront:

`|help>` 

```|help> sin
search: sin sinh sind sinc sinpi sincos sincosd sincospi asin using isinf asinh asind isinteger isinteractive thisind sign signed Signed signbit significand

  sin(x)

  Compute sine of x, where x is in radians.

  See also [sind], [sinpi], [sincos], [cis].

  ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

  sin(A::AbstractMatrix)

  Compute the matrix sine of a square matrix A.

  If A is symmetric or Hermitian, its eigendecomposition (eigen) is used to compute the sine. Otherwise, the sine is determined by calling exp.

  Examples
  ≡≡≡≡≡≡≡≡≡≡

  julia> sin(fill(1.0, (2,2)))
  2×2 Matrix{Float64}:
   0.454649  0.454649
   0.454649  0.454649
```

En jupyter simplemente escriben 

In [ ]:
?sin

Tambien se pueden ver todos los métodos asociados a una función en particular:

In [ ]:
@doc sin()

In [ ]:
methods(sin)

### Un ejemplo de documentación: ###

In [ ]:
"""Esta función duplica el argumento
admite tanto números como strings

### Examples
julia> eldoble(2.)

julia> 4.

julia> eldoble("dos")

julia> "dosdos"
"""
function eldoble(x::Float64)
    return 2*x
end
function eldoble(s::String)
    return s * s
end

#eldoble(2.0)

In [ ]:
eldoble("dos")

In [ ]:
?eldoble

In [ ]:
methods(eldoble)

In [ ]:
@which eldoble("dos")

In [ ]:
eldoble.([1.0;2.0])

In [ ]:
eldoble.(("uno", "dos"))

In [ ]:
using LinearAlgebra

In [ ]:
?LinearAlgebra

In [ ]:
apropos(LinearAlgebra)

In [ ]:
names(LinearAlgebra)

## Detalles del Código

El texto que nosotros escribimos es transformado sucesivamente hasta llegar al código que utiliza la máquina específica.

- `@code_lowered`: Código sin los aditivos que ayudan a la interfase humana.

- `@code_typed`: Los tipos de variables que se iran usando y sus conversiones.

- `@code_llvm`: Codigo compilado genérico.

- `@code_native`: Código asembler para la arquitectura de la computadora.

In [8]:
#x = 5
x = 3.0 + 1.0im
#x = [1 2 3]

3.0 + 1.0im

In [9]:
@code_lowered 1 * x

CodeInfo(
1 ─ %1 = Base.real(z)
│   %2 = x * %1
│   %3 = Base.imag(z)
│   %4 = x * %3
│   %5 = Base.Complex(%2, %4)
└──      return %5
)

In [10]:
@code_typed 1 * x

CodeInfo(
1 ─ %1 = Base.getfield(z, :re)::Float64
│   %2 = Base.sitofp(Float64, x)::Float64
│   %3 = Base.mul_float(%2, %1)::Float64
│   %4 = Base.getfield(z, :im)::Float64
│   %5 = Base.sitofp(Float64, x)::Float64
│   %6 = Base.mul_float(%5, %4)::Float64
│   %7 = %new(ComplexF64, %3, %6)::ComplexF64
└──      return %7
) => ComplexF64

In [11]:
@code_llvm 1 * x

;  @ complex.jl:333 within `*`
define void @"julia_*_1668"([2 x double]* noalias nocapture sret([2 x double]) %0, i64 signext %1, [2 x double]* nocapture nonnull readonly align 8 dereferenceable(16) %2) #0 {
top:
;  @ complex.jl:333 within `*` @ promotion.jl:380
; ┌ @ promotion.jl:350 within `promote`
; │┌ @ promotion.jl:327 within `_promote`
; ││┌ @ number.jl:7 within `convert`
; │││┌ @ float.jl:146 within `Float64`
      %3 = sitofp i64 %1 to double
; └└└└
;  @ complex.jl:333 within `*` @ promotion.jl:380 @ float.jl:405
  %4 = bitcast [2 x double]* %2 to <2 x double>*
  %5 = load <2 x double>, <2 x double>* %4, align 8
  %6 = insertelement <2 x double> poison, double %3, i32 0
  %7 = shufflevector <2 x double> %6, <2 x double> undef, <2 x i32> zeroinitializer
  %8 = fmul <2 x double> %5, %7
;  @ complex.jl:333 within `*`
  %9 = bitcast [2 x double]* %0 to <2 x double>*
  store <2 x double> %8, <2 x double>* %9, align 8
  ret void
}


In [13]:
@macroexpand @code_native 1 * x

:(InteractiveUtils.code_native(*, (Base.typesof)(1, x)))

In [14]:
@which InteractiveUtils.code_native

InteractiveUtils

In [15]:
methods(InteractiveUtils.code_native)

# 5 methods for generic function "code_native":
[1] code_native(io::IO, f) in InteractiveUtils at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/InteractiveUtils/src/codeview.jl:234
[2] code_native(::IO, ::Any, ::Symbol) in InteractiveUtils at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/InteractiveUtils/src/codeview.jl:245
[3] code_native(io::IO, f, types; syntax, debuginfo, binary) in InteractiveUtils at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/InteractiveUtils/src/codeview.jl:234
[4] code_native(f) in InteractiveUtils at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/InteractiveUtils/src/codeview.jl:243
[5] code_native(f, types; syntax, debuginfo, binary) in InteractiveUtils at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/InteractiveUtils/src/codeview.jl:243